In [11]:
import pandas as pd
from random import randint
from math import log

In [2]:
Df = pd.read_csv("./databreach.csv")
Df["Date"] = pd.to_datetime(Df["Date"])
Df["data_classes"] = [ i.split(",") for i in Df["data_classes"] ]

In [3]:
Df

,Date,Org,category,breach_count(Million),employee_count,data_classes
0,2017-10-01,Yahoo,web services,3000.0,8600.0,[Security questions and answers]
1,2018-03-01,UIDAI,AADHAR,1100.0,NaN,"[Names, Aadhar number, Bank account numbers]"
2,2014-01-01,Yahoo,web services,500.0,8600.0,"[Names, Email addresses, Phone numbers, Passwo..."
3,2013-06-01,MySpace,social media,360.0,150.0,"[Names, Usernames, Dates of birth]"
4,2018-06-01,Exactis,web services,340.0,NaN,"[Phone numbers, Physical addresses, Email addr..."
5,2018-05-01,Twitter,social media,330.0,5500.0,[Passwords]
6,2013-10-01,Adobe,e commerce,152.0,22516.0,"[Usernames, Email addresses, Passwords, Passwo..."
7,2018-02-01,MyFitnessPal,health,150.0,NaN,"[Email addresses, IP addresses, Usernames, Pas..."
8,2017-09-01,Equifax,web services,148.0,11200.0,"[Names, Physical addresses, Phone numbers, Dat..."
9,2014-03-01,ebay,e commerce,145.0,13300.0,"[Names, Email addresses, Physical addresses, P..."


### group by year

In [4]:
Df["year"] = Df.Date.dt.year
Df_year = pd.DataFrame(Df.groupby("year")["Date"].agg("count"))
Df = Df.drop(["year"], axis=1)


### generate mock data for missing employee counts

In [5]:
temp = []
for i in Df["employee_count"]:
    try:
        temp.append(int(i))
    except:
        temp.append(randint(0,1000))
Df["employee_count"] = temp

### scatter plot and exposure index

In [12]:
log(3)

1.0986122886681098

In [13]:
Df['exposure_index'] = [
                        log((Df["breach_count(Million)"][num])*len(i))
                        for num, i in enumerate(Df["data_classes"])
                        ]
Df_scatter = Df[["exposure_index", "employee_count", "category", "Org"]]


In [14]:
Df_scatter

,exposure_index,employee_count,category,Org
0,8.006368,8600,web services,Yahoo
1,8.101678,919,AADHAR,UIDAI
2,8.006368,8600,web services,Yahoo
3,6.984716,150,social media,MySpace
4,7.620705,263,web services,Exactis
5,5.799093,5500,social media,Twitter
6,6.410175,22516,e commerce,Adobe
7,6.396930,802,health,MyFitnessPal
8,6.943122,11200,web services,Equifax
9,6.768493,13300,e commerce,ebay


In [15]:
Df_scatter.to_csv("./final/scatter_data.csv")